In [262]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 0)

## Data Preprocessing

In this notebook, we will perform data preprocessing and feature extraction on the current datasets.

### Import Datasets

We will first import the datasets into the notebook and perform the necessary exposition

In [263]:
# Import datasets
df_listings = pd.read_csv('../data/raw/listings_raw.csv')
df_calendar = pd.read_csv('../data/raw/calendar_raw.csv')
df_reviews = pd.read_csv('../data/raw/reviews_raw.csv')

/var/folders/bj/_f2d36ln2cx_cj32b8ktzk780000gn/T/ipykernel_36831/624921297.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_calendar = pd.read_csv('../data/raw/calendar_raw.csv')


In [264]:
df_listings.head(5)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,13188,https://www.airbnb.com/rooms/13188,20240615011559,2024-06-15,city scrape,Garden level studio in ideal loc.,Garden level studio suite with garden patio - ...,The uber hip Main street area is a short walk ...,https://a0.muscache.com/pictures/8408188/e1af6...,51466,https://www.airbnb.com/users/show/51466,Emma,2009-11-04,"Vancouver, Canada",I love to travel with my family in comfort and...,within an hour,100%,100%,f,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,Riley Park,3,3,"['email', 'phone']",t,t,Neighborhood highlights,Riley Park,NaN,49.247730,-123.105090,Entire guest suite,Entire home/apt,4,1.0,1 bath,0.0,2.0,"[""Luggage dropoff allowed"", ""Ethernet connecti...",$160.00,2,180,1,2,1125,1125,2.0,1125.0,NaN,t,2,11,22,281,2024-06-15,304,44,9,2010-02-21,2024-06-12,4.85,4.88,4.85,4.87,4.93,4.89,4.81,24-157388,t,3,3,0,0,1.74
1,13358,https://www.airbnb.com/rooms/13358,20240615011559,2024-06-15,city scrape,Designer Downtown Vancouver suite,Iconic Electra Building.<br />A Vancouver Land...,2 blocks away from the shopping area of Robson...,https://a0.muscache.com/pictures/miso/Hosting-...,52116,https://www.airbnb.com/users/show/52116,Lynn,2009-11-07,"Vancouver, Canada",I am from Vancouver and in my free time I enjo...,within an hour,100%,99%,f,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,Downtown Vancouver,1,1,"['email', 'phone']",t,t,Neighborhood highlights,Downtown,NaN,49.281174,-123.125931,Entire condo,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""City skyline view"", ""Dishes and silverware"",...",$261.00,2,1125,1,2,1125,1125,2.0,1125.0,NaN,t,6,13,15,102,2024-06-15,518,60,5,2010-06-22,2024-05-30,4.68,4.75,4.81,4.83,4.79,4.91,4.65,24-159219,f,1,1,0,0,3.04
2,13490,https://www.airbnb.com/rooms/13490,20240615011559,2024-06-15,city scrape,Vancouver's best kept secret,This apartment rents for one month blocks of t...,"In the heart of Vancouver, this apartment has ...",https://a0.muscache.com/pictures/73394727/79d5...,52467,https://www.airbnb.com/users/show/52467,Iris,2009-11-08,"Vancouver, Canada",Hello! My husband Fred and I are actors living...,within an hour,100%,94%,t,https://a0.muscache.com/im/users/52467/profile...,https://a0.muscache.com/im/users/52467/profile...,Kensington-Cedar Cottage,3,4,"['email', 'phone']",t,t,Neighborhood highlights,Kensington-Cedar Cottage,NaN,49.256220,-123.066070,Entire rental unit,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""Luggage dropoff allowed"", ""Gym"", ""Ethernet c...",$225.00,30,180,30,30,180,180,30.0,180.0,NaN,t,0,0,13,244,2024-06-15,105,7,1,2011-06-02,2024-06-01,4.93,4.93,4.96,4.96,4.97

In [265]:
df_calendar.head(5)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,13188,2024-06-15,f,$120.00,NaN,2,1125
1,13188,2024-06-16,t,$120.00,NaN,2,1125
2,13188,2024-06-17,f,$120.00,NaN,2,1125
3,13188,2024-06-18,f,$120.00,NaN,2,1125
4,13188,2024-06-19,f,$120.00,NaN,2,1125


In [266]:
df_reviews.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,18270,201916,2011-03-17,184985,Matthew,"Great place, great location, great host."
1,13188,27060,2010-02-21,34595,Rebecca,"We had a wonderful time! The place is cozy, wa..."
2,13188,85604,2010-08-27,199181,Lillian,What a fabulous home away from home! The apar...
3,18270,219440,2011-04-06,99094,Jessica,I had a lovely and comfortable stay at Ran's....
4,18270,247798,2011-05-02,434670,Catherine,"Nice room, great location and friendly host!"


In [267]:
df_calendar[~df_calendar['adjusted_price'].isnull()]

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
1243555,715404243365272214,2024-06-15,f,"$1,166.00","$1,166.00",30,1095
1243556,715404243365272214,2024-06-16,f,"$1,166.00","$1,166.00",30,1095
1243557,715404243365272214,2024-06-17,f,"$1,166.00","$1,166.00",30,1095
1243558,715404243365272214,2024-06-18,f,"$1,166.00","$1,166.00",30,1095
1243559,715404243365272214,2024-06-19,f,"$1,166.00","$1,166.00",30,1095
...,...,...,...,...,...,...,...
1243915,715404243365272214,2025-06-10,f,"$1,166.00","$1,049.00",30,1095
1243916,715404243365272214,2025-06-11,f,"$1,166.00","$1,049.00",30,1095
1243917,715404243365272214,2025-06-12,f,"$1,166.00","$1,049.00",30,1095
1243918,715404243365272214,2025-06-13,f,"$1,166.00","$1,049.00",30,1095


In [268]:
# Prints the shape of the datasets
print(df_listings.shape)
print(df_calendar.shape)
print(df_reviews.shape)

(6233, 75)
(2275045, 7)
(272601, 6)


### Feature Summary and Types

A [summary](../data/data_summary.md) of the features and data types has been provided in the directory of the datasets, under the `data` folder.

### Checking Feature Types

It suffices to now check the types of each feature to determine which feature can be converted to a better representation

In [269]:
# Check variable types
print(df_listings.dtypes.to_string())

id                                                int64
listing_url                                      object
scrape_id                                         int64
last_scraped                                     object
source                                           object
name                                             object
description                                      object
neighborhood_overview                            object
picture_url                                      object
host_id                                           int64
host_url                                         object
host_name                                        object
host_since                                       object
host_location                                    object
host_about                                       object
host_response_time                               object
host_response_rate                               object
host_acceptance_rate                            

In [270]:
print(df_calendar.dtypes)

listing_id         int64
date              object
available         object
price             object
adjusted_price    object
minimum_nights     int64
maximum_nights     int64
dtype: object


In [271]:
print(df_reviews.dtypes)

listing_id        int64
id                int64
date             object
reviewer_id       int64
reviewer_name    object
comments         object
dtype: object


### Checking for NaN values

We should also check the number of NaN values that is in each feature

In [272]:
# Count NaN values
print(df_listings.isnull().sum().to_string())

id                                                 0
listing_url                                        0
scrape_id                                          0
last_scraped                                       0
source                                             0
name                                               0
description                                       87
neighborhood_overview                           2344
picture_url                                        0
host_id                                            0
host_url                                           0
host_name                                          0
host_since                                         0
host_location                                   1314
host_about                                      2963
host_response_time                              1097
host_response_rate                              1097
host_acceptance_rate                             877
host_is_superhost                             

In [273]:
print(df_calendar.isnull().sum())

listing_id              0
date                    0
available               0
price                   0
adjusted_price    2274680
minimum_nights          0
maximum_nights          0
dtype: int64


In [274]:
print(df_reviews.isnull().sum())

listing_id        0
id                0
date              0
reviewer_id       0
reviewer_name     1
comments         40
dtype: int64


### Number of Unique Values

We will also check the number of unique values for each features

In [275]:
print(df_listings.nunique().to_string())

id                                              6233
listing_url                                     6233
scrape_id                                          1
last_scraped                                       1
source                                             2
name                                            6166
description                                     5648
neighborhood_overview                           3390
picture_url                                     6116
host_id                                         4234
host_url                                        4234
host_name                                       2332
host_since                                      2703
host_location                                    113
host_about                                      2129
host_response_time                                 4
host_response_rate                                49
host_acceptance_rate                              82
host_is_superhost                             

In [276]:
print(df_calendar.nunique())

listing_id        6233
date               365
available            2
price              541
adjusted_price       2
minimum_nights      88
maximum_nights     152
dtype: int64


In [277]:
print(df_reviews.nunique())

listing_id         5235
id               272601
date               4480
reviewer_id      245704
reviewer_name     43121
comments         265718
dtype: int64


### Parsing Date-Based Features to Datetime

We notice that all of the features that contains information about time are being encoded as type objects.

However, to support better compability and for ease of use later during the data visualization process, we will convert all date-based features to type datetime.

In [278]:
# Parse datetime columns in listings
listings_datetime_cols = ['last_scraped', 'host_since', 'calendar_updated', 
                          'calendar_last_scraped', 'first_review', 'last_review']
df_listings[listings_datetime_cols] = df_listings[listings_datetime_cols].astype('datetime64[ns]', errors = 'raise')

# Parse datetime in calendar and reviews
df_calendar['date'] = df_calendar['date'].astype('datetime64[ns]', errors = 'raise')
df_reviews['date'] = df_reviews['date'].astype('datetime64[ns]', errors = 'raise')


### Convert Labels for Boolean Features

We noticed that all of the boolean features are classified with two labels: `f` and `t`. We will convert these values to `0` and `1` for clarity.

In [279]:
# Convert f/t labels to 0/1
listings_bool_cols = ['host_is_superhost', 'host_has_profile_pic',
                      'host_identity_verified', 'has_availability',
                      'instant_bookable']

df_listings[listings_bool_cols] = df_listings[listings_bool_cols].replace({'f' : 0, 't' : 1})
df_calendar['available'] = df_calendar['available'].replace({'f' : 0, 't' : 1})

/var/folders/bj/_f2d36ln2cx_cj32b8ktzk780000gn/T/ipykernel_36831/165103095.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_listings[listings_bool_cols] = df_listings[listings_bool_cols].replace({'f' : 0, 't' : 1})
/var/folders/bj/_f2d36ln2cx_cj32b8ktzk780000gn/T/ipykernel_36831/165103095.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_calendar['available'] = df_calendar['available'].replace({'f' : 0, 't' : 1})


In [280]:
df_listings

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,13188,https://www.airbnb.com/rooms/13188,20240615011559,2024-06-15,city scrape,Garden level studio in ideal loc.,Garden level studio suite with garden patio - ...,The uber hip Main street area is a short walk ...,https://a0.muscache.com/pictures/8408188/e1af6...,51466,https://www.airbnb.com/users/show/51466,Emma,2009-11-04,"Vancouver, Canada",I love to travel with my family in comfort and...,within an hour,100%,100%,0.0,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,Riley Park,3,3,"['email', 'phone']",1,1,Neighborhood highlights,Riley Park,NaN,49.247730,-123.105090,Entire guest suite,Entire home/apt,4,1.0,1 bath,0.0,2.0,"[""Luggage dropoff allowed"", ""Ethernet connecti...",$160.00,2,180,1,2,1125,1125,2.0,1125.0,NaT,1.0,2,11,22,281,2024-06-15,304,44,9,2010-02-21,2024-06-12,4.85,4.88,4.85,4.87,4.93,4.89,4.81,24-157388,1,3,3,0,0,1.74
1,13358,https://www.airbnb.com/rooms/13358,20240615011559,2024-06-15,city scrape,Designer Downtown Vancouver suite,Iconic Electra Building.<br />A Vancouver Land...,2 blocks away from the shopping area of Robson...,https://a0.muscache.com/pictures/miso/Hosting-...,52116,https://www.airbnb.com/users/show/52116,Lynn,2009-11-07,"Vancouver, Canada",I am from Vancouver and in my free time I enjo...,within an hour,100%,99%,0.0,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,Downtown Vancouver,1,1,"['email', 'phone']",1,1,Neighborhood highlights,Downtown,NaN,49.281174,-123.125931,Entire condo,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""City skyline view"", ""Dishes and silverware"",...",$261.00,2,1125,1,2,1125,1125,2.0,1125.0,NaT,1.0,6,13,15,102,2024-06-15,518,60,5,2010-06-22,2024-05-30,4.68,4.75,4.81,4.83,4.79,4.91,4.65,24-159219,0,1,1,0,0,3.04
2,13490,https://www.airbnb.com/rooms/13490,20240615011559,2024-06-15,city scrape,Vancouver's best kept secret,This apartment rents for one month blocks of t...,"In the heart of Vancouver, this apartment has ...",https://a0.muscache.com/pictures/73394727/79d5...,52467,https://www.airbnb.com/users/show/52467,Iris,2009-11-08,"Vancouver, Canada",Hello! My husband Fred and I are actors living...,within an hour,100%,94%,1.0,https://a0.muscache.com/im/users/52467/profile...,https://a0.muscache.com/im/users/52467/profile...,Kensington-Cedar Cottage,3,4,"['email', 'phone']",1,1,Neighborhood highlights,Kensington-Cedar Cottage,NaN,49.256220,-123.066070,Entire rental unit,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""Luggage dropoff allowed"", ""Gym"", ""Ethernet c...",$225.00,30,180,30,30,180,180,30.0,180.0,NaT,1.0,0,0,13,244,2024-06-15,105,7,1,2011-06-02,2024-06-01,4.93,4.93,4.

### Process Numerical Features

We will need to also convert some numerical features which have been intepretted as objects.

Some features will need to be reformatted to numbers, whereas some only needs a simple conversion to integer or float values

We will first convert two percentage-based columns: `host_response_rate` and `host_acceptance_rate`, to type `float` 

In [281]:
# Process percentages columns
df_listings['host_acceptance_rate'] = df_listings['host_acceptance_rate'].str.replace('%', '')
df_listings['host_response_rate'] = df_listings['host_response_rate'].str.replace('%', '')
df_listings[['host_response_rate', 'host_acceptance_rate']] = df_listings[['host_response_rate', 'host_acceptance_rate']].astype(float) / 100
df_listings[['host_response_rate', 'host_acceptance_rate']]

,host_response_rate,host_acceptance_rate
0,1.00,1.00
1,1.00,0.99
2,1.00,0.94
3,NaN,0.50
4,NaN,NaN
...,...,...
6228,NaN,NaN
6229,0.96,0.98
6230,1.00,1.00
6231,1.00,1.00


Next, we will convert the price representation of each datasets to type `float`

In [282]:
# Replace dollar signs and commas
price_pattern = r'\$|,'

df_listings['price'] = df_listings['price'].str.replace(price_pattern, '', regex = True)
df_calendar['price'] = df_calendar['price'].str.replace(price_pattern, '', regex = True)
df_calendar['adjusted_price'] = df_calendar['adjusted_price'].str.replace(price_pattern, '', regex = True)

# Convert to float representation
df_listings['price'] = df_listings['price'].astype(float)
df_calendar['price'] = df_calendar['price'].astype(float)
df_calendar['adjusted_price'] = df_calendar['adjusted_price'].astype(float)

### Dropping Unnecessary Columns

We notice that some of the columns are not relevant for further analysis. This includes:

- URLs of where the information is obtained

- Columns with all null values: `neighbourhood_group_cleansed` and `calendar_updated`

- Columns with only one label: `host_location`, `neighbourhood`

- Columns containing information about the scraper: `calendar_last_scraped` and `source`


Therefore, we can drop these columns from our datasets.

In [283]:
# Drop unncessary features from the datasets
df_listings.drop(['scrape_id', 'last_scraped', 'listing_url', 
                  'picture_url', 'host_url', 'host_thumbnail_url',
                  'host_picture_url', 'neighbourhood', 'neighbourhood_group_cleansed',
                  'calendar_updated', 'host_location', 'calendar_last_scraped', 
                  'source'], axis = 1, inplace = True)
df_listings.head(5)

,id,name,description,neighborhood_overview,host_id,host_name,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,13188,Garden level studio in ideal loc.,Garden level studio suite with garden patio - ...,The uber hip Main street area is a short walk ...,51466,Emma,2009-11-04,I love to travel with my family in comfort and...,within an hour,1.0,1.00,0.0,Riley Park,3,3,"['email', 'phone']",1,1,Riley Park,49.247730,-123.105090,Entire guest suite,Entire home/apt,4,1.0,1 bath,0.0,2.0,"[""Luggage dropoff allowed"", ""Ethernet connecti...",160.0,2,180,1,2,1125,1125,2.0,1125.0,1.0,2,11,22,281,304,44,9,2010-02-21,2024-06-12,4.85,4.88,4.85,4.87,4.93,4.89,4.81,24-157388,1,3,3,0,0,1.74
1,13358,Designer Downtown Vancouver suite,Iconic Electra Building.<br />A Vancouver Land...,2 blocks away from the shopping area of Robson...,52116,Lynn,2009-11-07,I am from Vancouver and in my free time I enjo...,within an hour,1.0,0.99,0.0,Downtown Vancouver,1,1,"['email', 'phone']",1,1,Downtown,49.281174,-123.125931,Entire condo,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""City skyline view"", ""Dishes and silverware"",...",261.0,2,1125,1,2,1125,1125,2.0,1125.0,1.0,6,13,15,102,518,60,5,2010-06-22,2024-05-30,4.68,4.75,4.81,4.83,4.79,4.91,4.65,24-159219,0,1,1,0,0,3.04
2,13490,Vancouver's best kept secret,This apartment rents for one month blocks of t...,"In the heart of Vancouver, this apartment has ...",52467,Iris,2009-11-08,Hello! My husband Fred and I are actors living...,within an hour,1.0,0.94,1.0,Kensington-Cedar Cottage,3,4,"['email', 'phone']",1,1,Kensington-Cedar Cottage,49.256220,-123.066070,Entire rental unit,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""Luggage dropoff allowed"", ""Gym"", ""Ethernet c...",225.0,30,180,30,30,180,180,30.0,180.0,1.0,0,0,13,244,105,7,1,2011-06-02,2024-06-01,4.93,4.93,4.96,4.96,4.97,4.80,4.89,NaN,0,1,1,0,0,0.66
3,14267,EcoLoft Vancouver,"The Ecoloft is located in the lovely, family r...",We live in the centre of the city of Vancouver...,56030,Peter & Alison,2009-11-20,We moved into this central Vancouver location ...,NaN,NaN,0.50,0.0,Kensington-Cedar Cottage,1,1,"['email', 'phone']",1,1,Kensington-Cedar Cottage,49.249220,-123.081390,Entire home,Entire home/apt,4,1.0,1 bath,1.0,2.0,"[""Dishes and silverware"", ""Shampoo"", ""Essentia...",180.0,30,365,30,30,365,365,30.0,365.0,1.0,0,27,56,56,33,0,0,2010-10-03,2021-07-14,4.76,4.84,4.68,4.90,4.68,4.77,4.71,21-156500,0,1,1,0,0,0.20
4,16254,Close to PNE/Hastings Park Garden level suite,2 Bedroom garden level guest suite.(lower leve...,"Good Eats, Cute Shops and all the personality ...",63238,Jason,2009-12-15,"Enjoys: skiing, snowboarding, mountain biking,...",NaN,NaN,NaN,0.0,Hastings-Sunrise,1,1,['phone'],1,1,Hastings-Sunrise,49.277210,-123.040860,Entire guest suite,Entire home/apt,4,1.0,1 bath,2.0,3.0,"[""Luggage dropoff allowed"", ""Dishes and silver...",800.0,5,31,5,5,1125,1125,5.0,1125.0,1.0,0,0,0,0,7,0,0,2018-08-04,2022-07-30,4.57,4.86,4.57,4.86,5.00,4.86,4.29,NaN,0,1,1,0,0,0.10


### Modifying Features

Lastly, we will modify some of the features from our dataset according to some translation in logic, as well as using our domain knowledege of Airbnb as well as the Vancouver region.

We do this for the following reasons:

- Reducing the cardinality of some categorical features
- Provide more identifiable and consistent labels

### Rename ID Feature

To ensure compatability with all of the other datasets, we will change the feature `id` in the listings dataset to `listing_id` just for continuity

In [284]:
# Rename ID
df_listings.rename({'id' : 'listing_id'}, axis = 1, inplace = True)
df_listings

,listing_id,name,description,neighborhood_overview,host_id,host_name,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,13188,Garden level studio in ideal loc.,Garden level studio suite with garden patio - ...,The uber hip Main street area is a short walk ...,51466,Emma,2009-11-04,I love to travel with my family in comfort and...,within an hour,1.00,1.00,0.0,Riley Park,3,3,"['email', 'phone']",1,1,Riley Park,49.247730,-123.105090,Entire guest suite,Entire home/apt,4,1.0,1 bath,0.0,2.0,"[""Luggage dropoff allowed"", ""Ethernet connecti...",160.0,2,180,1,2,1125,1125,2.0,1125.0,1.0,2,11,22,281,304,44,9,2010-02-21,2024-06-12,4.85,4.88,4.85,4.87,4.93,4.89,4.81,24-157388,1,3,3,0,0,1.74
1,13358,Designer Downtown Vancouver suite,Iconic Electra Building.<br />A Vancouver Land...,2 blocks away from the shopping area of Robson...,52116,Lynn,2009-11-07,I am from Vancouver and in my free time I enjo...,within an hour,1.00,0.99,0.0,Downtown Vancouver,1,1,"['email', 'phone']",1,1,Downtown,49.281174,-123.125931,Entire condo,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""City skyline view"", ""Dishes and silverware"",...",261.0,2,1125,1,2,1125,1125,2.0,1125.0,1.0,6,13,15,102,518,60,5,2010-06-22,2024-05-30,4.68,4.75,4.81,4.83,4.79,4.91,4.65,24-159219,0,1,1,0,0,3.04
2,13490,Vancouver's best kept secret,This apartment rents for one month blocks of t...,"In the heart of Vancouver, this apartment has ...",52467,Iris,2009-11-08,Hello! My husband Fred and I are actors living...,within an hour,1.00,0.94,1.0,Kensington-Cedar Cottage,3,4,"['email', 'phone']",1,1,Kensington-Cedar Cottage,49.256220,-123.066070,Entire rental unit,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""Luggage dropoff allowed"", ""Gym"", ""Ethernet c...",225.0,30,180,30,30,180,180,30.0,180.0,1.0,0,0,13,244,105,7,1,2011-06-02,2024-06-01,4.93,4.93,4.96,4.96,4.97,4.80,4.89,NaN,0,1,1,0,0,0.66
3,14267,EcoLoft Vancouver,"The Ecoloft is located in the lovely, family r...",We live in the centre of the city of Vancouver...,56030,Peter & Alison,2009-11-20,We moved into this central Vancouver location ...,NaN,NaN,0.50,0.0,Kensington-Cedar Cottage,1,1,"['email', 'phone']",1,1,Kensington-Cedar Cottage,49.249220,-123.081390,Entire home,Entire home/apt,4,1.0,1 bath,1.0,2.0,"[""Dishes and silverware"", ""Shampoo"", ""Essentia...",180.0,30,365,30,30,365,365,30.0,365.0,1.0,0,27,56,56,33,0,0,2010-10-03,2021-07-14,4.76,4.84,4.68,4.90,4.68,4.77,4.71,21-156500,0,1,1,0,0,0.20
4,16254,Close to PNE/Hastings Park Garden level suite,2 Bedroom garden level guest suite.(lower leve...,"Good Eats, Cute Shops and all the personality ...",63238,Jason,2009-12-15,"Enjoys: skiing, snowboarding, mountain biking,...",NaN,NaN,NaN,0.0,Hastings-Sunrise,1,1,['phone'],1,1,Hastings-Sunrise,49.277210,-123.040860,Entire guest suite,Entire home/apt,4,1.0,1 bath,2.0,3.0,"[""Luggage dropoff allowed"", ""Dishes and silver...",800.0,5,31,5,5,1125,1125,5.0,1125.0,1.0,0,0,0,0,7,0,0,2018-08-04,2022-07-30,4.57,4.86,4.57,4.86,5.00,4.86,4.29,NaN,0,1,

### Processing Host Verification Methods

We will first process the feature `host_verifications` first. Notice that while the format is stored as a list, there are only three different unique ways that a host can provide their identity from. These forms of verifications are `email`, `phone`, and `work_email`. 

Therefore, we can transform this column by creating three new boolean features called `host_verification_personal_email`, `host_verification_phone`, and `host_verification_work_email`, encoding whether the host has used the method for verification or not (1/0)

In [285]:
# Check for unique values
df_listings['host_verifications'].unique()

array(["['email', 'phone']", "['phone']",
       "['email', 'phone', 'work_email']", "['phone', 'work_email']",
       "['email']", '[]', "['email', 'work_email']"], dtype=object)

In [286]:
# Encode host_verification as boolean features in order of (personal_email, phone, work_email)
def process_host_verification(series):
    new_df = []
    for method_lst in series:
        in_order = [0,0,0]
        if 'email' in method_lst:
            in_order[0] = 1
        if 'phone' in method_lst:
            in_order[1] = 1
        if 'work_email' in method_lst:
            in_order[2] = 1
        new_df.append(in_order)
    
    return pd.DataFrame(new_df, index = series.index,
                        columns = ['host_verification_personal_email', 
                                    'host_verification_phone', 
                                    'host_verification_work_email'])

# Get output
output = df_listings['host_verifications'].pipe(process_host_verification)
df_listings.drop(['host_verifications'], axis = 1, inplace = True)
df_listings = pd.concat([df_listings, output], axis = 1)
df_listings

,listing_id,name,description,neighborhood_overview,host_id,host_name,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_verification_personal_email,host_verification_phone,host_verification_work_email
0,13188,Garden level studio in ideal loc.,Garden level studio suite with garden patio - ...,The uber hip Main street area is a short walk ...,51466,Emma,2009-11-04,I love to travel with my family in comfort and...,within an hour,1.00,1.00,0.0,Riley Park,3,3,1,1,Riley Park,49.247730,-123.105090,Entire guest suite,Entire home/apt,4,1.0,1 bath,0.0,2.0,"[""Luggage dropoff allowed"", ""Ethernet connecti...",160.0,2,180,1,2,1125,1125,2.0,1125.0,1.0,2,11,22,281,304,44,9,2010-02-21,2024-06-12,4.85,4.88,4.85,4.87,4.93,4.89,4.81,24-157388,1,3,3,0,0,1.74,1,1,0
1,13358,Designer Downtown Vancouver suite,Iconic Electra Building.<br />A Vancouver Land...,2 blocks away from the shopping area of Robson...,52116,Lynn,2009-11-07,I am from Vancouver and in my free time I enjo...,within an hour,1.00,0.99,0.0,Downtown Vancouver,1,1,1,1,Downtown,49.281174,-123.125931,Entire condo,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""City skyline view"", ""Dishes and silverware"",...",261.0,2,1125,1,2,1125,1125,2.0,1125.0,1.0,6,13,15,102,518,60,5,2010-06-22,2024-05-30,4.68,4.75,4.81,4.83,4.79,4.91,4.65,24-159219,0,1,1,0,0,3.04,1,1,0
2,13490,Vancouver's best kept secret,This apartment rents for one month blocks of t...,"In the heart of Vancouver, this apartment has ...",52467,Iris,2009-11-08,Hello! My husband Fred and I are actors living...,within an hour,1.00,0.94,1.0,Kensington-Cedar Cottage,3,4,1,1,Kensington-Cedar Cottage,49.256220,-123.066070,Entire rental unit,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""Luggage dropoff allowed"", ""Gym"", ""Ethernet c...",225.0,30,180,30,30,180,180,30.0,180.0,1.0,0,0,13,244,105,7,1,2011-06-02,2024-06-01,4.93,4.93,4.96,4.96,4.97,4.80,4.89,NaN,0,1,1,0,0,0.66,1,1,0
3,14267,EcoLoft Vancouver,"The Ecoloft is located in the lovely, family r...",We live in the centre of the city of Vancouver...,56030,Peter & Alison,2009-11-20,We moved into this central Vancouver location ...,NaN,NaN,0.50,0.0,Kensington-Cedar Cottage,1,1,1,1,Kensington-Cedar Cottage,49.249220,-123.081390,Entire home,Entire home/apt,4,1.0,1 bath,1.0,2.0,"[""Dishes and silverware"", ""Shampoo"", ""Essentia...",180.0,30,365,30,30,365,365,30.0,365.0,1.0,0,27,56,56,33,0,0,2010-10-03,2021-07-14,4.76,4.84,4.68,4.90,4.68,4.77,4.71,21-156500,0,1,1,0,0,0.20,1,1,0
4,16254,Close to PNE/Hastings Park Garden level suite,2 Bedroom garden level guest suite.(lower leve...,"Good Eats, Cute Shops and all the personality ...",63238,Jason,2009-12-15,"Enjoys: skiing, snowboarding, mountain biking,...",NaN,NaN,NaN,0.0,Hastings-Sunrise,1,1,1,1,Hastings-Sunrise,49.277210,-123.040860,Entire guest suite,Entire home/apt,4,1.0,1 bath,2.0,3.0,"[""Luggage dropoff allowed"", ""Dishes and silver...",800.0,5,31,5,5,1125,1125,5.0,1125.0,1.0,0,0,0,0,7,0,0,2018-08-04,2022-07-30,4.57,4.86,4.57,4.86,5.00,4.86,4.29,NaN,0,1,1,0

### Processing License Number

We decided that the actual license number is not as important as whether we have a license, and so this column would be more suited for a boolean feature. This would reduce the number of unique license values down to a binary feature.

We note the following values which counts as having a license and not having a license:

- NaN values &rarr; not having a license (0)
- Unique license ID or Exempt &rarr; having a license (1)



In [287]:
# Process license number
df_listings.loc[df_listings['license'].isnull(), 'license'] = 0
df_listings.loc[df_listings['license'] != 0, 'license'] = 1
df_listings['license'] = df_listings['license'].astype(int)
df_listings['license'].unique()

array([1, 0])

In [288]:
df_listings

,listing_id,name,description,neighborhood_overview,host_id,host_name,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_verification_personal_email,host_verification_phone,host_verification_work_email
0,13188,Garden level studio in ideal loc.,Garden level studio suite with garden patio - ...,The uber hip Main street area is a short walk ...,51466,Emma,2009-11-04,I love to travel with my family in comfort and...,within an hour,1.00,1.00,0.0,Riley Park,3,3,1,1,Riley Park,49.247730,-123.105090,Entire guest suite,Entire home/apt,4,1.0,1 bath,0.0,2.0,"[""Luggage dropoff allowed"", ""Ethernet connecti...",160.0,2,180,1,2,1125,1125,2.0,1125.0,1.0,2,11,22,281,304,44,9,2010-02-21,2024-06-12,4.85,4.88,4.85,4.87,4.93,4.89,4.81,1,1,3,3,0,0,1.74,1,1,0
1,13358,Designer Downtown Vancouver suite,Iconic Electra Building.<br />A Vancouver Land...,2 blocks away from the shopping area of Robson...,52116,Lynn,2009-11-07,I am from Vancouver and in my free time I enjo...,within an hour,1.00,0.99,0.0,Downtown Vancouver,1,1,1,1,Downtown,49.281174,-123.125931,Entire condo,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""City skyline view"", ""Dishes and silverware"",...",261.0,2,1125,1,2,1125,1125,2.0,1125.0,1.0,6,13,15,102,518,60,5,2010-06-22,2024-05-30,4.68,4.75,4.81,4.83,4.79,4.91,4.65,1,0,1,1,0,0,3.04,1,1,0
2,13490,Vancouver's best kept secret,This apartment rents for one month blocks of t...,"In the heart of Vancouver, this apartment has ...",52467,Iris,2009-11-08,Hello! My husband Fred and I are actors living...,within an hour,1.00,0.94,1.0,Kensington-Cedar Cottage,3,4,1,1,Kensington-Cedar Cottage,49.256220,-123.066070,Entire rental unit,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""Luggage dropoff allowed"", ""Gym"", ""Ethernet c...",225.0,30,180,30,30,180,180,30.0,180.0,1.0,0,0,13,244,105,7,1,2011-06-02,2024-06-01,4.93,4.93,4.96,4.96,4.97,4.80,4.89,0,0,1,1,0,0,0.66,1,1,0
3,14267,EcoLoft Vancouver,"The Ecoloft is located in the lovely, family r...",We live in the centre of the city of Vancouver...,56030,Peter & Alison,2009-11-20,We moved into this central Vancouver location ...,NaN,NaN,0.50,0.0,Kensington-Cedar Cottage,1,1,1,1,Kensington-Cedar Cottage,49.249220,-123.081390,Entire home,Entire home/apt,4,1.0,1 bath,1.0,2.0,"[""Dishes and silverware"", ""Shampoo"", ""Essentia...",180.0,30,365,30,30,365,365,30.0,365.0,1.0,0,27,56,56,33,0,0,2010-10-03,2021-07-14,4.76,4.84,4.68,4.90,4.68,4.77,4.71,1,0,1,1,0,0,0.20,1,1,0
4,16254,Close to PNE/Hastings Park Garden level suite,2 Bedroom garden level guest suite.(lower leve...,"Good Eats, Cute Shops and all the personality ...",63238,Jason,2009-12-15,"Enjoys: skiing, snowboarding, mountain biking,...",NaN,NaN,NaN,0.0,Hastings-Sunrise,1,1,1,1,Hastings-Sunrise,49.277210,-123.040860,Entire guest suite,Entire home/apt,4,1.0,1 bath,2.0,3.0,"[""Luggage dropoff allowed"", ""Dishes and silver...",800.0,5,31,5,5,1125,1125,5.0,1125.0,1.0,0,0,0,0,7,0,0,2018-08-04,2022-07-30,4.57,4.86,4.57,4.86,5.00,4.86,4.29,0,0,1,1,0,0,0.10,0,1,0
...,...,...,..

### Dropping Redundant Information

Next, we will drop some more features from the datasets which are considered **redundant**.  Redundant can be a very subjective word, and is open to interprettation. 

We will define features as redundant if the same information can be retrieved somewhere else in the datasets without losing the interprettability of the information.

From this definition, we immediately noticed that these features can be safetly dropped:

- `id` in reviews
    - review ids are unique and therefore non-repeating

    - Note that we cannot remove `listing_id` since it is the only way to join other datasets
    - `host_id` and `reviewer_id` cannot be dropped because we cannot guarantee that there will be no two listings / reviews made by the same host / reviewer

- `bathrooms_text`
    - The number of bathrooms has been encoded in `bathrooms`
    
    - Extra information such as `half` has been encoded as half decimals
    - The only information left is the keywords `shared` and `private`, but it is too vague for interpretting how many bathrooms are private or shared.

- `minimum_minimum_nights`	`maximum_minimum_nights`	`minimum_maximum_nights`	`maximum_maximum_nights`	`minimum_nights_avg_ntm`	` maximum_nights_avg_ntm`

    - Information regarding all of the aggregated informations are included in `df_calendar`

    - We will keep `minimum_nights` and `maximum_nights` since that indicates the most recent behaviour

- `host_listings_count`:
    - This column contains the number of active listings that are in or not in the current scrape

    - However, we can treat active listings that are not in the current scrape as inactive
    - Assuming the above, this information has already been included in `host_total_listings_count`

In [289]:
# Dropping redundant columns
df_listings.drop(['bathrooms_text', 'minimum_minimum_nights', 'maximum_minimum_nights',	
                  'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm',	
                  'maximum_nights_avg_ntm', 'host_listings_count'], axis = 1, inplace = True)

df_reviews.drop(['id'], axis = 1, inplace = True)

In [290]:
df_listings.isna().sum()

listing_id                                         0
name                                               0
description                                       87
neighborhood_overview                           2344
host_id                                            0
host_name                                          0
host_since                                         0
host_about                                      2963
host_response_time                              1097
host_response_rate                              1097
host_acceptance_rate                             877
host_is_superhost                                 41
host_neighbourhood                               312
host_total_listings_count                          0
host_has_profile_pic                               0
host_identity_verified                             0
neighbourhood_cleansed                             0
latitude                                           0
longitude                                     

### Fill NaN values for Listing Features

We can fill NaN values in some features with some reasonable justification and certainty. For NaN values that are hard to interpret we will just leave it as that:

- `host_is_superhost`: We assume that without information about being a superhost, that the listing is **not** a superhost 

- `has_availability`: We assume that without information about the listings' availability means that the listing is **not** available
- `adjusted_price`: We assume that without the adjusted price, the price remains the same and so we can fill it using values in `price`

In [291]:
# Fill NaN values in listing features
df_listings['host_is_superhost'] = df_listings['host_is_superhost'].fillna(0)
df_listings['has_availability'] = df_listings['has_availability'].fillna(0)
df_calendar['adjusted_price'] = df_calendar['adjusted_price'].fillna(df_calendar['price'])

### Handling NaN in Review Features

Filling NaN in review features such as `reviews per month`, requires a bit more nuance, since we can only be sure about filling the NaN values if the total review number is 0.

We cannot fill the rest of the review metrics such as review scores or review dates since there is no reviews, so we leave the NaN in place.

In [292]:
df_listings.loc[df_listings['number_of_reviews'] == 0, 'reviews_per_month'] = 0
df_listings

,listing_id,name,description,neighborhood_overview,host_id,host_name,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_verification_personal_email,host_verification_phone,host_verification_work_email
0,13188,Garden level studio in ideal loc.,Garden level studio suite with garden patio - ...,The uber hip Main street area is a short walk ...,51466,Emma,2009-11-04,I love to travel with my family in comfort and...,within an hour,1.00,1.00,0.0,Riley Park,3,1,1,Riley Park,49.247730,-123.105090,Entire guest suite,Entire home/apt,4,1.0,0.0,2.0,"[""Luggage dropoff allowed"", ""Ethernet connecti...",160.0,2,180,1.0,2,11,22,281,304,44,9,2010-02-21,2024-06-12,4.85,4.88,4.85,4.87,4.93,4.89,4.81,1,1,3,3,0,0,1.74,1,1,0
1,13358,Designer Downtown Vancouver suite,Iconic Electra Building.<br />A Vancouver Land...,2 blocks away from the shopping area of Robson...,52116,Lynn,2009-11-07,I am from Vancouver and in my free time I enjo...,within an hour,1.00,0.99,0.0,Downtown Vancouver,1,1,1,Downtown,49.281174,-123.125931,Entire condo,Entire home/apt,2,1.0,1.0,1.0,"[""City skyline view"", ""Dishes and silverware"",...",261.0,2,1125,1.0,6,13,15,102,518,60,5,2010-06-22,2024-05-30,4.68,4.75,4.81,4.83,4.79,4.91,4.65,1,0,1,1,0,0,3.04,1,1,0
2,13490,Vancouver's best kept secret,This apartment rents for one month blocks of t...,"In the heart of Vancouver, this apartment has ...",52467,Iris,2009-11-08,Hello! My husband Fred and I are actors living...,within an hour,1.00,0.94,1.0,Kensington-Cedar Cottage,4,1,1,Kensington-Cedar Cottage,49.256220,-123.066070,Entire rental unit,Entire home/apt,2,1.0,1.0,1.0,"[""Luggage dropoff allowed"", ""Gym"", ""Ethernet c...",225.0,30,180,1.0,0,0,13,244,105,7,1,2011-06-02,2024-06-01,4.93,4.93,4.96,4.96,4.97,4.80,4.89,0,0,1,1,0,0,0.66,1,1,0
3,14267,EcoLoft Vancouver,"The Ecoloft is located in the lovely, family r...",We live in the centre of the city of Vancouver...,56030,Peter & Alison,2009-11-20,We moved into this central Vancouver location ...,NaN,NaN,0.50,0.0,Kensington-Cedar Cottage,1,1,1,Kensington-Cedar Cottage,49.249220,-123.081390,Entire home,Entire home/apt,4,1.0,1.0,2.0,"[""Dishes and silverware"", ""Shampoo"", ""Essentia...",180.0,30,365,1.0,0,27,56,56,33,0,0,2010-10-03,2021-07-14,4.76,4.84,4.68,4.90,4.68,4.77,4.71,1,0,1,1,0,0,0.20,1,1,0
4,16254,Close to PNE/Hastings Park Garden level suite,2 Bedroom garden level guest suite.(lower leve...,"Good Eats, Cute Shops and all the personality ...",63238,Jason,2009-12-15,"Enjoys: skiing, snowboarding, mountain biking,...",NaN,NaN,NaN,0.0,Hastings-Sunrise,1,1,1,Hastings-Sunrise,49.277210,-123.040860,Entire guest suite,Entire home/apt,4,1.0,2.0,3.0,"[""Luggage dropoff allowed"", ""Dishes and silver...",800.0,5,31,1.0,0,0,0,0,7,0,0,2018-08-04,2022-07-30,4.57,4.86,4.57,4.86,5.00,4.86,4.29,0,0,1,1,0,0,0.10,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6228,1178437395127036835,Luxury apartment in Vancouver,"Newly furnished, this beautiful apartment in t...",NaN,332013608,Elizabet

### Convert Features to Type Category

For some of the object features that has covered the range of all possible values, we can classify them as type `category` to reduce the memory usage of the data.

These features include `neighbourhood_cleansed` and `room_type`

In [293]:
df_listings['neighbourhood_cleansed'] = df_listings['neighbourhood_cleansed'].astype('category')
df_listings['room_type'] = df_listings['room_type'].astype('category')

### Removing Line Breaks in Reviews

Lastly, to avoid any importing problems with pandas, we will replace all instances terminators `<br/>`, `\r` and line break `\n` from any text fields within the datasets.

In [294]:
line_breaks_pattern = r"(<br/>|\n|\r)"

# Iterate through columns and remove line breaks
for col in df_listings.columns:
    if df_listings[col].dtypes == 'object':
        df_listings[col] = df_listings[col].str.replace(line_breaks_pattern, '', regex = True)

# Remove line breaks in comments
df_reviews['comments'] = df_reviews['comments'].str.replace(line_breaks_pattern, '', regex = True)
df_reviews

,listing_id,date,reviewer_id,reviewer_name,comments
0,18270,2011-03-17,184985,Matthew,"Great place, great location, great host."
1,13188,2010-02-21,34595,Rebecca,"We had a wonderful time! The place is cozy, wa..."
2,13188,2010-08-27,199181,Lillian,What a fabulous home away from home! The apar...
3,18270,2011-04-06,99094,Jessica,I had a lovely and comfortable stay at Ran's....
4,18270,2011-05-02,434670,Catherine,"Nice room, great location and friendly host!"
...,...,...,...,...,...
272596,1167658394866031187,2024-06-01,75090412,Georgina,"Loved the place. Easy to check-in, Driana was ..."
272597,1167658394866031187,2024-06-08,115858985,Jason,Driana and her husband were the ideal hosts! ...
272598,1168245100183046449,2024-06-11,62180956,Nathaniel,Javier was extremely hospitable to my family. ...
272599,1168449186334658361,2024-06-09,42273592,Agi,Wir waren 2 Nächte in das Haus. Wir haben uns ...


### Export Datasets

Having done our preprocessing steps, we will export our datasets for further analysis

In [295]:
# Export datasets
df_listings.to_csv('../data/processed/listings_processed.csv', index = False)
df_calendar.to_csv('../data/processed/calendar_processed.csv', index = False)
df_reviews.to_csv('../data/processed/reviews_processed.csv', index = False)
